In [1]:
import torch
import numpy as np

## Initialization with tensor

In [2]:
nsamples = 100
nnodes = 1000
edge_freq = 0.004
cliq_edge_freq = 0.114
hi_mut_freq = 0.5
hi_node = nnodes*3/4
group_labels = ['Subtype 1']*(nsamples//2) + ['Subtype 2']*(nsamples//2)
feature_names = ['Subnetwork 1', 'Subnetwork 2', 'High mut source', 'High mut target', 
                 'Random 1', 'Random 2', 
                 'Self loop', 'Intercept']
node_names = ['{}'.format(i) for i in range(1,nnodes+1)]
sample_names = ['{}'.format(i) for i in range(1,nsamples+1)]

rand_mut_freq = 0.015

In [3]:
degrees = [0]*nnodes
edges = []
features = [] #(11) cliq1, cliq2, hi_mut_source, hi_mut_target, rand1, rand2, rand3, rand4, rand5, self_loop, intercept
for i in range(nnodes-1):
    for j in range(i+1,nnodes):
        if ((i<100 and j<100) and np.random.random()<cliq_edge_freq) or np.random.random()<edge_freq:
            edges.append([i,j])
            edges.append([j,i])
            features.append([0,0,0,0,np.random.random(),np.random.random(),0,1])
            features.append([0,0,0,0,np.random.random(),np.random.random(),0,1])
            if (i<50 and j<50):
                features[-2][0] = 1
                features[-1][0] = 1
            if (i>=50 and i<100 and j>=50 and j<100):
                features[-2][1] = 1
                features[-1][1] = 1
            if i == nnodes-1:
                features[-2][2] = 1
                features[-1][3] = 1
            if j == nnodes-1:
                features[-2][3] = 1
                features[-1][2] = 1
            degrees[i] += 1
            degrees[j] += 1
            

for i in range(nnodes):
    edges.append([i,i])
    features.append([0,0,0,0,np.random.random(),np.random.random(),1,1])

P_init = []
for p in range(nsamples):
    p_init = []
    for i in range(nnodes):
        freq=0
        if p == i:
            freq = 1
        elif i == hi_node:
            freq = hi_mut_freq
        elif i<100:
            if (max(p,i)<50 or min(p,i)>=50):
                freq = 0.015
            else:
                freq = 0.000
        else:
            freq = rand_mut_freq

        if np.random.random() < freq:
            p_init.append(1.0)
        else:
            p_init.append(0.0)

    P_init.append(p_init)

In [4]:
P_init = torch.tensor(P_init, requires_grad = True)
features = torch.tensor(features, requires_grad = True, dtype=torch.float64)

## loss function

In [5]:
def extract_distinct_labels(group_labels):
    '''
    compute labels dictionary
    {
        'type 1': [sample_start_position_1, sample_end_position_1],
        'type 2': [sample_start_position_2, sample_end_position_2],
        ...
    }
    '''
    all_labels = {}
    for i, label in enumerate(group_labels):
        if label not in all_labels:
            all_labels[label] = [i]
        if i == len(group_labels)-1 or group_labels[i+1] not in all_labels:
            all_labels[label].append(i+1)
    return all_labels

In [6]:
def calculate_centers_SRW(P, all_labels):
    '''
    compute group centers, adapted from SRW centroid() function
    
    return a tensor with group centroids
    if we have k groups and n nodes, then C.shape = (k, n)
    '''
    C = torch.zeros(size=(len(all_labels), P.shape[1]))
    count = 0
    for label in all_labels:
        start, end = all_labels[label]
        C[count,:] = torch.sum(P[start: end, :], axis = 0) / (end - start)
        count += 1
    return C

In [7]:
def label_to_id(all_labels):
    '''
    mapping label string to id
     {
        'type 1': 0,
        'type 2': 1,
        ...
    }
    
    '''
    label_id = {}
    count = 0
    for label in all_labels:
        label_id[label] = count
        count += 1
    return label_id

In [8]:
def loss(lambda_value, w, beta, P, group_labels, all_labels, is_train):
    '''
    compute loss function adapted from SRW cost_func_WMW()
    '''
    '''l1 norm'''
    loss_value = lambda_value * torch.norm(w, p=1)
    accuracy = 0.0
    
    '''retrieve centers'''
    C = calculate_centers_SRW(P, all_labels)
    
    '''retrieve ids for group labels'''
    label_id = label_to_id(all_labels)
    
    '''necessary intermediate value for computing loss'''
    P_dot_CT = torch.matmul(P, C.T)
    C_dot_CT = torch.matmul(C, C.T)
    P_dot_PT = torch.matmul(P, P.T)
    
    '''simply copy from SRW cost_func_WMW()'''
    for u in range(P.shape[0]):
        x_u = torch.tensor(-2.0)
        i = label_id[group_labels[u]]
        start, end = all_labels[group_labels[u]]
        group_sample = end - start
        if is_train == False:
            coeff = (group_sample / (group_sample - 1)) ** 2
        else:
            coeff = 1.0
        dist_ui = coeff *( P_dot_PT[u,u] -2 * P_dot_CT[u, i] + C_dot_CT[i,i])
        for label in label_id:
            if label != group_labels[u]:
                j = label_id[label]
                x_u_tmp = dist_ui -(P_dot_PT[u,u] - 2*P_dot_CT[u,j] + C_dot_CT[j,j])
                if x_u_tmp > x_u:
                    x_u = x_u_tmp
        '''if correctly classified, increase accuracy'''
        if x_u < 0.0:
            accuracy += 1.0
        loss_value += 1. / (1+torch.exp(-x_u / beta))
    return loss_value, accuracy / P.shape[0]

## Model class implement with pytorch
* train with simple gradient descent
* add validation module 

In [9]:
class SRW_pytorch:
    def __init__(self, n_iter, lambda_value, beta, group_labels, features, edges, P, nnodes, rst_prob, lr):
        self.n_iter = n_iter
        self.lambda_value = lambda_value
        self.beta = beta
        self.group_labels = group_labels
        self.features = features
        self.edges = edges
        self.P = P
        self.nnodes = nnodes
        self.rst_prob = rst_prob
        self.lr = lr
        self.is_train = False
        self.w = None
        self.Q_final = None
        
    def train(self):
        w = torch.normal(mean = 0, std = 1, size=(features.shape[1],1), requires_grad = True, dtype=torch.float64)
        for i in range(n_iter):
            '''compute edge strength, sigmoid activation'''
            strength = 1.0 / (1.0 + torch.exp(-torch.matmul(self.features, w)))

            '''create transition matrix Q'''
            Q = torch.zeros(size=(self.nnodes, self.nnodes))
            for j in range(strength.shape[0]):
                Q[self.edges[j][0], self.edges[j][1]] = strength[j, 0]

            '''normalize Q'''
            Q = Q / (torch.sum(Q, axis = 1) + 1e-8).reshape(-1,1)
            self.Q_final = Q
            
            '''noramlize P'''
            P_init = self.P / (torch.sum(self.P, axis = 1) + 1e-8).reshape(-1,1)

            '''create P matrix for random walk'''
            P = torch.detach(P_init)
            P.requires_grad = True

            '''for test, only peform random walk 30 times (should be enough to converge)''' 
            for j in range(30):
                P = (1-self.rst_prob) * (torch.matmul(P,Q)) + self.rst_prob * P_init

            '''compute loss and backward()'''
            all_labels = extract_distinct_labels(self.group_labels)
            loss_value, accuracy = loss(self.lambda_value, w, self.beta, P, self.group_labels, all_labels, self.is_train)
            loss_value.backward(retain_graph = True)

            '''simple GD optimization'''
            with torch.no_grad():
                w -= self.lr * w.grad
                self.w = w
            
            '''zero grad'''
            w.grad.zero_()
            print("[%d/%d] training loss: %.4f\t training accuracy: %.4f" %(i+1, n_iter, loss_value.data, accuracy))
        self.is_train = True
    
    def validation(self, P_validation, group_labels_validation):
        if self.is_train == False:
            print("You should train the model first")
            return
        
        P = torch.detach(P_validation)
        Q_final = self.Q_final
        for j in range(30):
                P = (1-self.rst_prob) * (torch.matmul(P,Q_final)) + self.rst_prob * P_validation
        
        all_labels = extract_distinct_labels(group_labels_validation)
        loss_value, accuracy = loss(self.lambda_value, self.w, self.beta, P, group_labels_validation, all_labels, self.is_train)
        
        return loss_value, accuracy

In [10]:
n_iter = 10
lambda_value = 0.1
beta = 2e-4
rst_prob = 0.3
lr = 1.0
solver = SRW_pytorch(n_iter, lambda_value, beta, group_labels, features, edges, P_init, nnodes, rst_prob, lr)

In [ ]:
solver.train()

[1/10] training loss: 46.6808	 training accuracy: 0.5700
[2/10] training loss: 45.7408	 training accuracy: 0.5800
[3/10] training loss: 45.4708	 training accuracy: 0.5800
[4/10] training loss: 45.3632	 training accuracy: 0.5800
[5/10] training loss: 45.2260	 training accuracy: 0.5800
[6/10] training loss: 45.0049	 training accuracy: 0.5800
[7/10] training loss: 44.5474	 training accuracy: 0.5900


In [ ]:
loss_value, accuracy = solver.validation(P_init, group_labels)
print("validation loss: %.4f\t validation_accuracy: %.4f" %(loss.data, accuracy))